<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%204/Assignment_part_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Use of Spacy**

Now, we integrated spaCy with RandomForestClassifier. SpaCy, with its advanced natural language processing capabilities, allowed us to effectively parse and understand the linguistic features of our text data. This enriched data was then fed into RandomForestClassifier, a robust model known for handling complex classification tasks. 

While RandomForest excels in many scenarios, we found that its traditional strength in numerical data doesn't fully translate to the nuanced requirements of text data, presenting a unique challenge in accurately classifying text difficulty. Unfortunately, the result was not good enough.

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
df_train = pd.read_csv("https://raw.githubusercontent.com/Oglo/Project-DSML/main/Data/training_data.csv").dropna()
df_test = pd.read_csv("https://raw.githubusercontent.com/Oglo/Project-DSML/main/Data/unlabelled_test_data.csv").dropna()
df_final = pd.read_csv("https://raw.githubusercontent.com/Oglo/Project-DSML/main/Data/sample_submission.csv").dropna()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import spacy

nlp = spacy.load('fr_core_news_sm')

def extract_features(text):
    doc = nlp(text)
    num_sentences = len(list(doc.sents))
    num_tokens = len(doc)
    avg_sentence_length = num_tokens / num_sentences if num_sentences > 0 else 0
    lexical_diversity = len(set(token.text.lower() for token in doc)) / num_tokens if num_tokens > 0 else 0
    return [avg_sentence_length, lexical_diversity]

df_train['features'] = df_train['sentence'].apply(extract_features)

features_df = pd.DataFrame(df_train['features'].tolist(), columns=['avg_sentence_length', 'lexical_diversity'])
X = features_df
y = df_train['difficulty'] 

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred_val = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred_val)
print("Accuracy:", accuracy)

df_test['features'] = df_test['sentence'].apply(extract_features)
test_features_df = pd.DataFrame(df_test['features'].tolist(), columns=['avg_sentence_length', 'lexical_diversity'])
X_test = test_features_df

y_pred_test = model.predict(X_test)

df_final['difficulty'] = y_pred_test 

df_final['difficulty'] = df_final['difficulty'].map(difficulty_mapping)